<a href="https://colab.research.google.com/github/a-agmon/interviewdata/blob/main/Billing_Analytics_HS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Please press Play on each of the following cells that prepare the data for the question.

In [1]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib

In [2]:
import duckdb
import pandas as pd
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [6]:
%%sql
drop table if exists invoices;
drop table if exists payment_methods;

,Success


In [7]:
%%sql
CREATE TABLE payment_methods (
    id INT,
    method_name VARCHAR(50),
    description VARCHAR(255),
    is_active BOOLEAN
);

-- Insert sample payment methods
INSERT INTO payment_methods (id, method_name, description, is_active)
VALUES
    (1, 'Credit Card', 'Payment via credit card', true),
    (2, 'Bank Transfer', 'Direct bank transfer', true),
    (3, 'PayPal', 'Payment through PayPal service', true),
    (4, 'Cash', 'Cash payment on delivery', false);

-- Create the invoices table
CREATE TABLE invoices (
    id INT ,
    customer_name VARCHAR(100),
    invoice_date DATE,
    amount DECIMAL(10, 2),
    status VARCHAR(20),
    payment_method_id INT,
);

INSERT INTO invoices (id, customer_name, invoice_date, amount, status, payment_method_id)
VALUES
    (1, 'Acme Corp', '2024-09-01', 1500.00, 'Paid', 1),
    (2, 'TechSolutions Inc', '2024-09-03', 2750.50, 'Pending', 2),
    (3, 'Global Traders', '2024-09-05', 980.25, 'Paid', 3),
    (4, 'Innovative Systems', '2024-09-07', 3200.00, 'Overdue', 1),
    (5, 'QuickServe Logistics', '2024-09-10', 1875.75, 'Paid', 2),
    (6, 'Stellar Productions', '2024-09-12', 4500.00, 'Pending', 3),
    (7, 'EcoFriendly Solutions', '2024-09-15', 2100.50, 'Paid', 1),
    (8, 'MegaCorp Industries', '2024-09-18', 5500.00, 'Overdue', 2),
    (9, 'SmartTech Innovations', '2024-09-20', 1950.25, 'Pending', 3),
    (10, 'Sunrise Enterprises', '2024-09-22', 3750.00, 'Paid', -1),
    (11, 'Green Energy Co', '2024-09-25', 2200.00, 'Paid', 1),
    (12, 'DataDrive Analytics', '2024-09-28', 3100.75, 'Pending', 2),
    (13, 'Quantum Computing Ltd', '2024-10-01', 6500.00, 'Overdue', 3),
    (14, 'Nano Technologies', '2024-10-03', 1800.50, 'Paid', 1),
    (15, 'Cloud Nine Services', '2024-10-05', 2900.25, 'Pending', 2),
    (16, 'Artificial Intelligence Inc', '2024-10-08', 4200.00, 'Paid', 3),
    (17, 'Cyber Security Solutions', '2024-10-10', 3500.75, 'Overdue', 1),
    (18, 'Blockchain Innovations', '2024-10-12', 5100.00, 'Pending', 2),
    (19, 'Virtual Reality Labs', '2024-10-15', 2700.50, 'Paid', 3),
    (20, 'Future Tech Enterprises', '2024-10-18', 3900.25, 'Pending', -1);

,Success


### 2. This is the invoices table ###


In [12]:
%%sql
select * from invoices limit 5

,id,customer_name,invoice_date,amount,status,payment_method_id
0,1,Acme Corp,2024-09-01,1500.00,Paid,1
1,2,TechSolutions Inc,2024-09-03,2750.50,Pending,2
2,3,Global Traders,2024-09-05,980.25,Paid,3
3,4,Innovative Systems,2024-09-07,3200.00,Overdue,1
4,5,QuickServe Logistics,2024-09-10,1875.75,Paid,2


### 2. This is the payment method table ###

In [13]:
%%sql
select * from payment_methods limit 5

,id,method_name,description,is_active
0,1,Credit Card,Payment via credit card,True
1,2,Bank Transfer,Direct bank transfer,True
2,3,PayPal,Payment through PayPal service,True
3,4,Cash,Cash payment on delivery,False


## Question 1:
The following query shows all invoices and payment methods. However, for some reason we dont see all invoices. Can you please help us understand why?

In [18]:
%%sql
select *
from invoices i join payment_methods p on i.payment_method_id == p.id

,id,customer_name,invoice_date,amount,status,payment_method_id,id_1,method_name,description,is_active
0,1,Acme Corp,2024-09-01,1500.00,Paid,1,1,Credit Card,Payment via credit card,True
1,2,TechSolutions Inc,2024-09-03,2750.50,Pending,2,2,Bank Transfer,Direct bank transfer,True
2,3,Global Traders,2024-09-05,980.25,Paid,3,3,PayPal,Payment through PayPal service,True
3,4,Innovative Systems,2024-09-07,3200.00,Overdue,1,1,Credit Card,Payment via credit card,True
4,5,QuickServe Logistics,2024-09-10,1875.75,Paid,2,2,Bank Transfer,Direct bank transfer,True
5,6,Stellar Productions,2024-09-12,4500.00,Pending,3,3,PayPal,Payment through PayPal service,True
6,7,EcoFriendly Solutions,2024-09-15,2100.50,Paid,1,1,Credit Card,Payment via credit card,True
7,8,MegaCorp Industries,2024-09-18,5500.00,Overdue,2,2,Bank Transfer,Direct bank transfer,True
8,9,SmartTech Innovations,2024-09-20,1950.25,Pending,3,3,PayPal,Payment through PayPal service,True
9,11,Green Energy Co,2024-09-25,2200.00,Paid,1,1,Credit Card,Payment via credit card,True


###Question 2###

Please write a query that shows the number of invoices and total amount for each payment method

In [20]:
%%sql
select 1 + 1

,(1 + 1)
0,2
